# Generating the AUC tables



In [11]:
%load_ext autoreload

%autoreload 2
from utils import parse_table, read_csv
import pandas as pd
import numpy as np

from pretty import plot_kde, ColorTheme


def get_scatter(in_distr, ood, type, all_tasks): 
    global likelihood_generated, lid_generated
    global likelihood_in, lid_in
    global likelihood_ood, lid_ood
    print(all_tasks.keys())
    print(all_tasks[type].keys())
    in_vs_out = all_tasks[type][in_distr][ood]
    likelihood_generated = in_vs_out[in_vs_out['name'] == 'generated']['log-likelihood'].values
    lid_generated = in_vs_out[in_vs_out['name'] == 'generated']['LID'].values
    likelihood_in = in_vs_out[in_vs_out['name'] == 'test']['log-likelihood'].values
    lid_in = in_vs_out[in_vs_out['name'] == 'test']['LID'].values
    likelihood_ood = in_vs_out[in_vs_out['name'] == 'ood']['log-likelihood'].values
    lid_ood = in_vs_out[in_vs_out['name'] == 'ood']['LID'].values

print_mapping = {
    'fashion-mnist': 'FMNIST',
    'mnist': 'MNIST',
    'emnist': 'EMNIST',
    'omniglot': 'Omniglot',
    'celeba-small': 'CelebA',
    'svhn': 'SVHN',
    'cifar10': 'CIFAR10',
    'cifar100': 'CIFAR100',
    'tiny-imagenet': 'Tiny',
}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from roc_analysis import get_roc_graph, get_convex_hull, get_auc
from tqdm import tqdm

def generate_table(compute_limit, all_tasks, verbose=1):
    np.random.seed(111)
    latex_table = """
    \\begin{tabularx}{\\textwidth}{l*{4}{Y}}
    \\toprule
    OOD Task Type & \\multicolumn{2}{c}{$A$-gen (vs) $B$}$^\\mathsection$ & \\multicolumn{2}{c}{$A$ (vs) $B$}$^\dagger$ \\\\
    \\cmidrule(r){2-3} \\cmidrule(lr){4-5}
    \\leftcell{Dataset Pair \\\\$A$ (and) $B$}&  \makecell{{\scriptsize (AUC-ROC)} \\\\ Na\\"ive Likelihood} &  \makecell{{\\scriptsize (AUC-ROC)} \\\\ LID OOD (Ours)}&  \\makecell{{\\scriptsize (AUC-ROC)} \\\\ Na\\"ive Likelihood} &  \\makecell{{\\scriptsize (AUC-ROC)} \\\\ LID OOD (Ours)}  \\\\
    \\midrule
    """

    
    for iii, type in enumerate(['grayscale', 'rgb']):

        if verbose > 1:
            print("---------- Grayscale ----------" if type == 'grayscale' else "---------- RGB ----------")
        pairs = []
        for in_distr in all_tasks[type].keys():
            for ood in all_tasks[type].keys():
                if in_distr != ood:
                    pairs.append((in_distr, ood))
        rng = pairs
        if verbose == 1:
            rng = tqdm(rng, desc="Grayscale data pairs")
        for in_distr, ood in rng:
            if verbose > 1:
                print(f"\n\n== {in_distr} vs {ood} ==")
            get_scatter(in_distr, ood, type, all_tasks)
            latex_table += f"{{\\scriptsize {print_mapping[in_distr]} (and) {print_mapping[ood]}}} & "
            x_naive, y_naive = get_roc_graph(
                pos_x = likelihood_generated,
                neg_x = likelihood_ood,
                verbose=0,
            )
            x_curve, y_curve = get_convex_hull(x_naive, y_naive)
            naive_auc = get_auc(x_curve, y_curve)
            formatted_auc1 = "{:.3f}".format(naive_auc)
            if verbose > 1:
                print(f"(Generated) Naive likelihood AUC = {formatted_auc1}")
            x_ours, y_ours = get_roc_graph(
                pos_x = likelihood_generated,
                pos_y = lid_generated,
                neg_x = likelihood_ood,
                neg_y = lid_ood,
                compute_limit=compute_limit,
                verbose=0,
            )
            x_curve, y_curve = get_convex_hull(x_ours, y_ours)
            our_auc = max(naive_auc, get_auc(x_curve, y_curve))
            formatted_auc2 = "{:.3f}".format(our_auc)
            improvement = (our_auc - naive_auc)/naive_auc * 100
            if improvement > 10:
                latex_table += f"$\\mathbf{{{formatted_auc1}}}$ & $\\mathbf{{{formatted_auc2}}}$ &"
            else:
                latex_table += f"${formatted_auc1}$ & ${formatted_auc2}$ &"
            if verbose > 1:
                print("(Generated) LID-OOD AUC = {:.3f}".format(our_auc))
                
                
            x_naive, y_naive = get_roc_graph(
                pos_x = likelihood_in,
                neg_x = likelihood_ood,
                verbose=0,
            )
            x_curve, y_curve = get_convex_hull(x_naive, y_naive)
            naive_auc = get_auc(x_curve, y_curve)
            formatted_auc1 = "{:.3f}".format(naive_auc)
            if verbose > 1:
                print(f"Naive likelihood AUC = {formatted_auc1}")
            x_ours, y_ours = get_roc_graph(
                pos_x = likelihood_in,
                pos_y = lid_in,
                neg_x = likelihood_ood,
                neg_y = lid_ood,
                compute_limit=compute_limit,
                verbose=0,
            )
            x_curve, y_curve = get_convex_hull(x_ours, y_ours)
            
            our_auc = max(naive_auc, get_auc(x_curve, y_curve))
            formatted_auc2 = "{:.3f}".format(our_auc)
            improvement = (our_auc - naive_auc)/naive_auc * 100
            if improvement > 10:
                latex_table += f"$\\mathbf{{{formatted_auc1}}}$ & $\\mathbf{{{formatted_auc2}}}$ \\\\ \n"
            else:
                latex_table += f"${formatted_auc1}$ & ${formatted_auc2}$ \\\\ \n"
            if verbose > 1:
                print("(Generated) LID-OOD AUC = {:.3f}".format(our_auc))
                
        if iii == 0:
            latex_table += '\\midrule \n\n'

    
    latex_table += """
    \\bottomrule
    \\end{tabularx}
    """          
    return latex_table

In [13]:
df = read_csv('grayscale_LID_vs_likelihood_diffusion_lpca.csv')
all_tasks_grayscale = parse_table(df)
df = read_csv('rgb_LID_vs_likelihood_diffusion_lpca.csv')
all_tasks_rgb = parse_table(df)

all_tasks = {
    'grayscale': all_tasks_grayscale,
    'rgb': all_tasks_rgb,
}

5/7: 100%|██████████| 5/5 [00:14<00:00,  2.98s/it]


In [14]:


latex_table = generate_table(100000, all_tasks=all_tasks, verbose=2)
with open("generated_tabulars/all_boosting_diffusion.tex", 'w') as f:
    f.write(latex_table)
    

---------- Grayscale ----------


== mnist vs mnist ==
dict_keys(['grayscale', 'rgb'])
dict_keys(['mnist', 'emnist', 'fashion-mnist', 'omniglot'])


KeyError: 'mnist'